# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |   Camila Daniela Zapata Castañeda|
| **Fecha**      |   17 de Febrero del 2025|
| **Expediente** |   745624|

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Es la manera más sencilla de dividir el tu data frame en las secciones de train y test

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [55]:
import pandas as pd
import numpy as np

In [56]:
data = pd.read_excel("Motor Trend Car Road Tests.xlsx")
data.head(5)

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [57]:
datadrop = data.drop(columns = ["model"])

In [58]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from scipy import stats

In [59]:
x = datadrop.drop(columns = ["mpg"])
x.shape

(32, 10)

In [60]:
y = pd.DataFrame()
y["mpg"] = data["mpg"]

In [61]:
lista =[]

In [62]:
for i in range(len(y)):
    
    test_x = x.iloc[[i], :]
    train_x = x.drop(index=i) 
    
    test_y = y.iloc[[i]]
    train_y = y[y.index != i]
    
    scaler = StandardScaler()
    scaler.fit(train_x)
    
    X_std_test = scaler.transform(test_x)
    X_std_train = scaler.transform(train_x)
    
    model = LinearRegression()
    
    model = model.fit(X_std_train, train_y)
    y_pred_test = model.predict(X_std_test)
    
    mse = mean_squared_error(test_y, y_pred_test)
    
    lista.append(mse)

In [63]:
mse_array = np.array(lista)
mse_array.mean()

12.181558006901936

In [64]:
mse_array.std()

17.067399871888576

Interpreta.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [125]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [126]:
X = pd.DataFrame(X)
X.columns=["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"]
X.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [130]:
y = pd.DataFrame(y)
y.columns=["MedHouseVal"]
X["MedHouseVal"] = y

In [131]:
data = X.copy()
data.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [132]:
b = data.index
random = np.random.choice(b, round(len(data)), replace = False)

In [133]:
bar = data.iloc[random, :]
bar.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
4909,2.3190,21.0,3.590164,1.024590,1505.0,4.112022,34.02,-118.27,0.972
14309,1.6953,35.0,3.743902,1.097561,691.0,4.213415,32.71,-117.13,0.813
8453,2.8512,25.0,3.431694,1.009107,1337.0,2.435337,33.91,-118.35,2.728
11971,2.7600,26.0,5.520871,1.085299,1720.0,3.121597,34.01,-117.45,0.952
7559,2.7813,33.0,4.799331,1.076923,1298.0,4.341137,33.90,-118.20,1.051


In [134]:
A = len(y)
A = len(y)/10
A = int(A)

In [144]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
4909,2.3190,21.0,3.590164,1.024590,1505.0,4.112022,34.02,-118.27
14309,1.6953,35.0,3.743902,1.097561,691.0,4.213415,32.71,-117.13
8453,2.8512,25.0,3.431694,1.009107,1337.0,2.435337,33.91,-118.35
11971,2.7600,26.0,5.520871,1.085299,1720.0,3.121597,34.01,-117.45
7559,2.7813,33.0,4.799331,1.076923,1298.0,4.341137,33.90,-118.20
...,...,...,...,...,...,...,...,...
17826,5.7695,3.0,4.826263,1.057576,2522.0,2.547475,37.38,-121.89
14038,2.7852,37.0,3.754098,1.075820,955.0,1.956967,32.75,-117.14
18276,7.3561,35.0,6.697143,1.051429,512.0,2.925714,37.34,-122.07
6107,4.2019,37.0,5.164659,1.024096,1047.0,4.204819,34.11,-117.90


In [169]:
lista3 = []

In [166]:
y

,MedHouseVal
4909,0.97200
14309,0.81300
8453,2.72800
11971,0.95200
7559,1.05100
...,...
17826,2.71400
14038,1.29200
18276,5.00001
6107,1.40100


In [170]:
X = bar.drop(columns = ["MedHouseVal"])
y = bar.drop(columns = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"])

for i in range (10):
    y_test = y.iloc[A*i : A*(i+1), :]
    y_train = y.drop(index = y_test.index)
    
    X_test = X.iloc[A*i : A*(i+1), :]
    X_train = X.drop(index = X_test.index)
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_std_test = scaler.transform(X_test)
    X_std_train = scaler.transform(X_train)
    
    model = LinearRegression()
    
    model = model.fit(X_std_train, y_train)
    y_pred_test = model.predict(X_std_test)
    
    mse = mean_squared_error(y_test, y_pred_test)
    
    lista3.append(mse)

In [171]:
lista3

[0.54202303571985,
 0.5212546279138819,
 0.5043058523125518,
 0.49208912871864496,
 0.542065730361302,
 0.5711098367521249,
 0.5288046807556273,
 0.49153251335467435,
 0.5620388804437133,
 0.5234696702341426]

In [173]:
mse_array = np.array(lista3)
mse_array

array([0.54202304, 0.52125463, 0.50430585, 0.49208913, 0.54206573,
       0.57110984, 0.52880468, 0.49153251, 0.56203888, 0.52346967])

In [174]:
mse_array.mean()

0.5278693956566514

In [175]:
mse_array.std()

0.025804990737533756

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3